# Network Intrusion Detection System (NIDS)
## Notebook 2: Preprocessing & Feature Engineering

**Team Member:** Member 2  
**Dataset:** CIC-IDS2017 (Multi-class Classification)  
**Date:** November 24, 2025  

**Objectives:**
1. Load combined dataset from Notebook 1
2. Handle missing values
3. Handle outliers (based on Member 1's analysis)
4. Feature engineering (create 2-3 new features)
5. Encode categorical variables
6. Feature scaling (StandardScaler)
7. Train-test split (70-30 stratified)
8. Save processed data for modeling

---

## 1. Import Libraries

In [1]:
# Data manipulation
import numpy as np
import pandas as pd

# Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
import glob
import os
import warnings
warnings.filterwarnings('ignore')

#Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print(" Libraries imported successfully!")

 Libraries imported successfully!


In [ ]:
# ============================================================================
# LOCAL OUTPUT SAVER (for Colab VS Code Extension)
# ============================================================================
# This ensures all outputs are saved to your local machine
# ============================================================================

import os
from pathlib import Path

# Detect if running on Colab
IN_COLAB = 'COLAB_GPU' in os.environ or 'google.colab' in str(get_ipython())

if IN_COLAB:
    # Mount Google Drive
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        
        # Set base path to your local project in Drive
        # IMPORTANT: Update this path to match your Google Drive structure
        BASE_PATH = '/content/drive/MyDrive/MLCEProject'
        
        # Create output directories if they don't exist
        for dir_name in ['outputs', 'models', 'data']:
            Path(f'{BASE_PATH}/{dir_name}').mkdir(parents=True, exist_ok=True)
        
        print("✓ Google Drive mounted")
        print(f"✓ Base path: {BASE_PATH}")
        print(f"✓ Outputs will save to: {BASE_PATH}/outputs")
        print(f"✓ Models will save to: {BASE_PATH}/models")
        print(f"✓ Data will save to: {BASE_PATH}/data")
        
    except Exception as e:
        print(f"⚠️  Could not mount Drive: {e}")
        print("Using Colab local storage (will not sync automatically)")
        BASE_PATH = '/content'
else:
    # Running locally - use relative paths
    BASE_PATH = '..'
    print("✓ Running locally")
    print("✓ Using relative paths (../outputs, ../models, ../data)")

# Helper functions for saving with correct paths
def get_output_path(filename):
    """Get correct path for output file"""
    return f"{BASE_PATH}/outputs/{filename}"

def get_model_path(filename):
    """Get correct path for model file"""
    return f"{BASE_PATH}/models/{filename}"

def get_data_path(filename):
    """Get correct path for data file"""
    return f"{BASE_PATH}/data/{filename}"

print("\n✓ Local save helper ready!")
print("\nUse these functions to save files:")
print("  - get_output_path('plot.png')  → saves to outputs/")
print("  - get_model_path('model.pkl')  → saves to models/")
print("  - get_data_path('data.csv')    → saves to data/\n")


---
## 2. Load Combined Dataset

Load the combined CIC-IDS2017 dataset (same as Notebook 1).

In [2]:
from pathlib import Path
# Dataset Verification Function (from Notebook 1)
def verify_dataset(path):
    """Verify dataset existence and validity"""
    path = Path(path)
    if not path.exists():
        return False, []
        
    csv_files = list(path.glob('*.csv'))
    if not csv_files:
        return False, []
        
    # Check total size (should be > 100MB for full dataset)
    total_size = sum(f.stat().st_size for f in csv_files) / (1024**2)
    if total_size < 10:
        print(f"Warning: Dataset seems too small ({total_size:.2f} MB)")
        
    return True, csv_files

# OPTIMIZATION: Function to reduce memory usage (Robust Version)
def optimize_dtypes(df):
    # Handle duplicate columns if any (keep first)
    df = df.loc[:, ~df.columns.duplicated()]
    
    # Downcast floats to float32
    float_cols = df.select_dtypes(include=['float64']).columns
    for col in float_cols:
        try:
            df[col] = df[col].astype('float32')
        except Exception:
            pass
            
    # Downcast integers
    int_cols = df.select_dtypes(include=['int64']).columns
    for col in int_cols:
        try:
            df[col] = pd.to_numeric(df[col], downcast='integer')
        except Exception:
            pass
            
    return df

# Load all CSV files and combine with OPTIMIZATIONS
data_path = Path('../data/CICIDS2017/')

# Verify dataset first
valid, csv_files = verify_dataset(data_path)

if not valid:
    raise FileNotFoundError(f"Dataset not found or invalid in {data_path}. Please run Notebook 1 first.")

print(f"Found {len(csv_files)} CSV files. Loading with PyArrow engine...")

dfs = []
failed_files = []

for file in csv_files:
    try:
        print(f"Reading: {file.name}...", end=" ")
        # Try using pyarrow engine for faster reading
        try:
            df_temp = pd.read_csv(file, engine='pyarrow')
        except:
            df_temp = pd.read_csv(file) # Fallback
        
        # Optimize memory immediately
        df_temp = optimize_dtypes(df_temp)
        
        dfs.append(df_temp)
        print(f" Shape: {df_temp.shape} ✓")
        
    except Exception as e:
        print(f" FAILED! Error: {e}")
        failed_files.append(file)

if not dfs:
    raise RuntimeError("No data was loaded successfully!")

print("\nCombining dataframes...")
df = pd.concat(dfs, ignore_index=True)

# OPTIMIZATION: Drop unnecessary columns early (saves memory)
unnecessary_cols = ['Source IP', 'Destination IP', 'Timestamp']
df = df.drop(columns=[c for c in unnecessary_cols if c in df.columns], errors='ignore')

# Clean column names
df.columns = df.columns.str.strip()

# CLEANUP: Fix encoding issues in 'Label' column (same as Notebook 1)
if 'Label' in df.columns:
    print("Cleaning 'Label' column...", end=" ")
    df['Label'] = df['Label'].str.replace('\ufffd', '-', regex=False)
    df['Label'] = df['Label'].str.strip()
    print("✓")

print(f"\n Dataset loaded")
print(f"Shape: {df.shape}")
print(f"Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


Found 8 CSV files. Loading with PyArrow engine...
Reading: Tuesday-WorkingHours.pcap_ISCX.csv...  Shape: (445909, 78) ✓
Reading: Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv...  Shape: (288602, 78) ✓
Reading: Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv...  Shape: (170366, 78) ✓
Reading: Monday-WorkingHours.pcap_ISCX.csv...  Shape: (529918, 78) ✓
Reading: Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv...  Shape: (225745, 78) ✓
Reading: Friday-WorkingHours-Morning.pcap_ISCX.csv...  Shape: (191033, 78) ✓
Reading: Wednesday-workingHours.pcap_ISCX.csv...  Shape: (692703, 78) ✓
Reading: Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv...  Shape: (286467, 78) ✓

Combining dataframes...
Cleaning 'Label' column... ✓

 Dataset loaded
Shape: (2830743, 78)
Memory: 807.94 MB


In [3]:
# Identify target column
target_col = ' Label' if ' Label' in df.columns else 'Label'
print(f"Target column: '{target_col}'")
print(f"Classes: {df[target_col].unique()}")

Target column: 'Label'
Classes: ['BENIGN' 'FTP-Patator' 'SSH-Patator' 'Infiltration'
 'Web Attack - Brute Force' 'Web Attack - XSS'
 'Web Attack - Sql Injection' 'DDoS' 'Bot' 'DoS slowloris'
 'DoS Slowhttptest' 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed' 'PortScan']


---
## 3. Handle Missing Values

Check for missing values and handle them appropriately.

In [4]:
# Check missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
}).sort_values('Missing Count', ascending=False)

cols_with_missing = missing_df[missing_df['Missing Count'] > 0]

print("Columns with missing values:")
if len(cols_with_missing) > 0:
    print(cols_with_missing)
else:
    print("No missing values found!")

Columns with missing values:
              Missing Count  Percentage
Flow Bytes/s           1358      0.0480


In [5]:
# Handle missing values
# Strategy: Drop columns with >50% missing, impute others with median

if len(cols_with_missing) > 0:
    # Drop columns with >50% missing
    high_missing_cols = cols_with_missing[cols_with_missing['Percentage'] > 50].index.tolist()
    if len(high_missing_cols) > 0:
        print(f"\nDropping {len(high_missing_cols)} columns with >50% missing values")
        df = df.drop(columns=high_missing_cols)
    
    # Impute remaining missing values with median
    remaining_missing = df.columns[df.isnull().any()].tolist()
    if target_col in remaining_missing:
        remaining_missing.remove(target_col)
    
    if len(remaining_missing) > 0:
        print(f"\nImputing {len(remaining_missing)} columns with median")
        
        # OPTIMIZATION: Pre-compute all medians at once (faster)
        medians = df[remaining_missing].median()
        for col in remaining_missing:
            df[col] = df[col].fillna(medians[col])
    
    print(f"\n✓ Missing values handled")
    print(f"Remaining missing values: {df.isnull().sum().sum()}")
else:
    print("\n✓ No missing values to handle")



Imputing 1 columns with median

✓ Missing values handled
Remaining missing values: 0


---
## 4. Handle Outliers

Based on Member 1's outlier analysis, handle outliers using capping method (Winsorization).

In [6]:
# Load outlier summary from Notebook 1
outlier_path = Path('../outputs/outlier_summary.csv')

if outlier_path.exists():
    outlier_summary = pd.read_csv(outlier_path)
    
    # Select features with >10% outliers for capping
    features_to_cap = outlier_summary[outlier_summary['Percentage'] > 10]['Feature'].tolist()
    
    print(f"Features with >10% outliers: {len(features_to_cap)}")
    print(f"\nCapping outliers at 1st and 99th percentiles (VECTORIZED)...")
    
    # OPTIMIZATION: Vectorized capping (5-10x faster)
    # Compute percentiles for all features at once
    p01 = df[features_to_cap].quantile(0.01)
    p99 = df[features_to_cap].quantile(0.99)
    
    # Clip all features at once
    df[features_to_cap] = df[features_to_cap].clip(lower=p01, upper=p99, axis=1)
    
    print(f"\n✓ Outliers capped for {len(features_to_cap)} features")
else:
    print("\nWarning: outlier_summary.csv not found. Skipping outlier capping.")
    print("Please run Notebook 1 to generate outlier analysis.")


Features with >10% outliers: 49

Capping outliers at 1st and 99th percentiles (VECTORIZED)...

✓ Outliers capped for 49 features


---
## 5. Feature Engineering

Create 2-3 new features based on domain knowledge.

In [7]:
# Feature Engineering: Create new features
print("Creating new features...\n")

# Feature 1: Packet Rate (packets per second)
if 'Flow Duration' in df.columns and 'Total Fwd Packets' in df.columns:
    df['Packet_Rate'] = df['Total Fwd Packets'] / (df['Flow Duration'] + 1)  # +1 to avoid division by zero
    print(" Created: Packet_Rate = Total Fwd Packets / Flow Duration")

# Feature 2: Byte Rate (bytes per second)
if 'Flow Duration' in df.columns and 'Total Length of Fwd Packets' in df.columns:
    df['Byte_Rate'] = df['Total Length of Fwd Packets'] / (df['Flow Duration'] + 1)
    print(" Created: Byte_Rate = Total Length of Fwd Packets / Flow Duration")

# Feature 3: Packet Size Ratio (forward/backward)
if 'Total Length of Fwd Packets' in df.columns and 'Total Length of Bwd Packets' in df.columns:
    df['Packet_Size_Ratio'] = df['Total Length of Fwd Packets'] / (df['Total Length of Bwd Packets'] + 1)
    print(" Created: Packet_Size_Ratio = Total Length of Fwd Packets / Total Length of Bwd Packets")

print(f"\nNew feature count: 3")
print(f"Total features: {df.shape[1]}")

Creating new features...

 Created: Packet_Rate = Total Fwd Packets / Flow Duration
 Created: Byte_Rate = Total Length of Fwd Packets / Flow Duration
 Created: Packet_Size_Ratio = Total Length of Fwd Packets / Total Length of Bwd Packets

New feature count: 3
Total features: 81


---
## 6. Encode Categorical Variables

** Professor Requirement #3 (Partial): Prepare I/O variables**

In [8]:
# Encode target variable (attack labels)
label_encoder = LabelEncoder()
df['Label_Encoded'] = label_encoder.fit_transform(df[target_col])

# Save label mapping
# Create mapping with standard Python types for JSON serialization
label_mapping = {str(label): int(code) for label, code in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}
print("Label Encoding Mapping:")
for label, code in label_mapping.items():
    print(f"  {code}: {label}")

# Save mapping to file
import json
with open(get_output_path('label_mapping.json', 'w') as f:
    json.dump(label_mapping, f, indent=2)

print("\n Label mapping saved: outputs/label_mapping.json")

Label Encoding Mapping:
  0: BENIGN
  1: Bot
  2: DDoS
  3: DoS GoldenEye
  4: DoS Hulk
  5: DoS Slowhttptest
  6: DoS slowloris
  7: FTP-Patator
  8: Heartbleed
  9: Infiltration
  10: PortScan
  11: SSH-Patator
  12: Web Attack - Brute Force
  13: Web Attack - Sql Injection
  14: Web Attack - XSS

 Label mapping saved: outputs/label_mapping.json


In [9]:
# Check for other categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Remove target column from categorical list
if target_col in categorical_cols:
    categorical_cols.remove(target_col)

print(f"Other categorical columns: {categorical_cols}")

# Handle if any exist (e.g., protocol types)
if len(categorical_cols) > 0:
    print("\nEncoding categorical features...")
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        print(f"   Encoded: {col}")
else:
    print("\n No additional categorical columns to encode")

Other categorical columns: []

 No additional categorical columns to encode


---
## 7. Separate Features and Target

** Professor Requirement #3: Identify input-output variables**

In [10]:
# Define X (features) and y (target)
# Exclude original label column and encoded label column
exclude_cols = [target_col, 'Label_Encoded']
feature_cols = [col for col in df.columns if col not in exclude_cols]

X = df[feature_cols]
y = df['Label_Encoded']

print("="*70)
print("INPUT-OUTPUT VARIABLES DEFINITION")
print("="*70)
print(f"Input Variables (X): {X.shape[1]} features")
print(f"Output Variable (y): {y.name} (multi-class, {y.nunique()} classes)")
print(f"Total Samples: {len(X):,}")
print("="*70)

print(f"\nFeature names (first 10):")
for i, col in enumerate(X.columns[:10], 1):
    print(f"  {i:2d}. {col}")
print(f"  ... ({X.shape[1] - 10} more features)")

# OPTIMIZATION: Clear memory
import gc
del df
gc.collect()
print("Memory cleared: df deleted")


INPUT-OUTPUT VARIABLES DEFINITION
Input Variables (X): 80 features
Output Variable (y): Label_Encoded (multi-class, 15 classes)
Total Samples: 2,830,743

Feature names (first 10):
   1. Destination Port
   2. Flow Duration
   3. Total Fwd Packets
   4. Total Backward Packets
   5. Total Length of Fwd Packets
   6. Total Length of Bwd Packets
   7. Fwd Packet Length Max
   8. Fwd Packet Length Min
   9. Fwd Packet Length Mean
  10. Fwd Packet Length Std
  ... (70 more features)
Memory cleared: df deleted


---
## 8. Train-Test Split

** Professor Requirement #4: Split data into train and test**

Perform 70-30 stratified split to maintain class balance.

In [ ]:
# Perform stratified train-test split (70-30)
print("Performing train-test split (70-30 stratified)...\n")

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    test_size=0.3,
    random_state=42,
    stratify=y  # Maintain class distribution
)

print("="*70)
print("TRAIN-TEST SPLIT SUMMARY")
print("="*70)
print(f"Training set:   {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set:       {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"Features:       {X_train.shape[1]}")
print("="*70)

# Verify stratification
print("\nClass distribution verification:")
print("\nOriginal:")
print(y.value_counts(normalize=True).sort_index())
print("\nTraining:")
print(y_train.value_counts(normalize=True).sort_index())
print("\nTest:")
print(y_test.value_counts(normalize=True).sort_index())

print("\n Stratified split maintains class distribution")

# OPTIMIZATION: Clear memory
del X, y
gc.collect()
print("Memory cleared: X and y deleted")

# Save feature names for later use
feature_names = X_train.columns.tolist()
print(f"\nSaved {len(feature_names)} feature names for CSV export")


---
## 9. Feature Scaling

Apply StandardScaler to normalize features (zero mean, unit variance).

**Important:** Fit scaler on training data only, transform both train and test.

In [12]:
# Handle inf/NaN values BEFORE scaling (critical for preventing crashes)
print("Handling inf/NaN values before scaling...")
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)

# Fill NaN with median (simple and effective)
X_train = X_train.fillna(X_train.median())
X_test = X_test.fillna(X_train.median())  # Use training median for test
print(f"✓ Handled inf/NaN values\n")

# Initialize StandardScaler
scaler = StandardScaler()

print("Applying feature scaling (StandardScaler) with CHUNKING...\n")

# OPTIMIZATION: Use smaller chunks for safety
chunk_size = 50000  # Reduced from 100000
total_samples = X_train.shape[0]

print(f"Fitting scaler on {total_samples:,} samples in chunks of {chunk_size:,}...")

# Fit on training data in chunks
for i in range(0, total_samples, chunk_size):
    end = min(i + chunk_size, total_samples)
    chunk = X_train.iloc[i:end]
    scaler.partial_fit(chunk)
    if (i // chunk_size) % 10 == 0:
        print(f"  Processed {end:,}/{total_samples:,} samples...")
    del chunk
    gc.collect()

print("✓ Scaler fitted!\n")

# Transform training data in chunks
print("Transforming training data...")
X_train_scaled_list = []
for i in range(0, total_samples, chunk_size):
    end = min(i + chunk_size, total_samples)
    chunk = X_train.iloc[i:end]
    X_train_scaled_list.append(scaler.transform(chunk))
    del chunk
    if (i // chunk_size) % 10 == 0:
        gc.collect()

X_train_scaled = np.vstack(X_train_scaled_list)
del X_train_scaled_list, X_train
gc.collect()
print("✓ Training data scaled\n")

# Transform test data in chunks
print("Transforming test data...")
total_test = X_test.shape[0]
X_test_scaled_list = []
for i in range(0, total_test, chunk_size):
    end = min(i + chunk_size, total_test)
    chunk = X_test.iloc[i:end]
    X_test_scaled_list.append(scaler.transform(chunk))
    del chunk
    if (i // chunk_size) % 10 == 0:
        gc.collect()

X_test_scaled = np.vstack(X_test_scaled_list)
del X_test_scaled_list, X_test
gc.collect()
print("✓ Test data scaled\n")

print(f"✓ Feature scaling complete")
print(f"Scaled training shape: {X_train_scaled.shape}")
print(f"Scaled test shape: {X_test_scaled.shape}")


Handling inf/NaN values before scaling...
✓ Handled inf/NaN values

Applying feature scaling (StandardScaler) with CHUNKING...

Fitting scaler on 1,981,520 samples in chunks of 50,000...
  Processed 50,000/1,981,520 samples...
  Processed 550,000/1,981,520 samples...
  Processed 1,050,000/1,981,520 samples...
  Processed 1,550,000/1,981,520 samples...
✓ Scaler fitted!

Transforming training data...
✓ Training data scaled

Transforming test data...
✓ Test data scaled

✓ Feature scaling complete
Scaled training shape: (1981520, 80)
Scaled test shape: (849223, 80)


---
## 10. Save Processed Data

Save train and test sets for use in Notebook 3 (Model Training).

In [ ]:
# Save to CSV files
print("Saving processed data to CSV files...\n")

# Convert numpy arrays to DataFrames first
print("Converting to DataFrames...")
# Convert to DataFrames with original feature names
X_train_df = pd.DataFrame(X_train_scaled, columns=feature_names)
X_test_df = pd.DataFrame(X_test_scaled, columns=feature_names)
print("✓\n")

# OPTIMIZATION: Use float32 and fixed precision to speed up writing
print("Saving X_train.csv (this may take a few minutes)...", end=" ")
X_train_df.astype('float32').to_csv(get_data_path('X_train.csv', index=False, float_format='%.4f')
print("✓")

print("Saving X_test.csv...", end=" ")
X_test_df.astype('float32').to_csv(get_data_path('X_test.csv', index=False, float_format='%.4f')
print("✓")

print("Saving y_train.csv...", end=" ")
y_train.to_csv(get_data_path('y_train.csv', index=False, header=['Label_Encoded'])
print("✓")

print("Saving y_test.csv...", end=" ")
y_test.to_csv(get_data_path('y_test.csv', index=False, header=['Label_Encoded'])
print("✓")

print("\n✓ All processed data saved successfully!")
print(f"\nSaved files:")
print(f"  - data/X_train.csv: {X_train_df.shape[0]:,} rows × {X_train_df.shape[1]} columns")
print(f"  - data/X_test.csv: {X_test_df.shape[0]:,} rows × {X_test_df.shape[1]} columns")
print(f"  - data/y_train.csv: {len(y_train):,} rows")
print(f"  - data/y_test.csv: {len(y_test):,} rows")


---
## 11. Preprocessing Summary

### Actions Performed:
1.  Loaded CIC-IDS2017 dataset ([fill number] records)
2.  Handled missing values (dropped/imputed as needed)
3.  Capped outliers at 1st and 99th percentiles ([fill number] features)
4.  Created 3 new features (Packet_Rate, Byte_Rate, Packet_Size_Ratio)
5.  Encoded target variable (multi-class: [fill number] classes)
6.  Defined X ([fill number] features) and y (Label_Encoded)
7.  Performed 70-30 stratified train-test split
8.  Applied StandardScaler (fit on train, transform train+test)
9.  Saved processed data to CSV files

### Dataset Summary:
- **Training samples:** [fill from output]
- **Test samples:** [fill from output]
- **Features:** [fill from output]
- **Classes:** [fill from label_mapping]
- **Class balance:** Maintained through stratified splitting

### Files Generated:
- `data/X_train.csv` - Scaled training features
- `data/X_test.csv` - Scaled test features
- `data/y_train.csv` - Training labels
- `data/y_test.csv` - Test labels
- `outputs/label_mapping.json` - Label encoding reference

---

**Next Steps for Member 3 (Modeling):**
1. Load preprocessed data from CSV files
2. Train 3+ models (Logistic Regression, SVC, PCA+LogReg)
3. Generate parity plots for train and test
4. Compute classification metrics
5. Create confusion matrices and ROC curves

---

**Proceed to:** `03_model_training_evaluation.ipynb`